In [5]:
import sys
from pathlib import Path
import import_ipynb
import numpy as np
import gmsh
from scipy.sparse.linalg import spsolve
from analytical_solution import potential_and_gradient

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    from pre_processing import domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results, error_evaluation
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Modules imports were successful!


# Project 1: Coaxial Problem 

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# `setTypeProblem()`

In [6]:
def setTypeProblem(element_type, lc):
    mesh_data = {}
    gmsh.initialize()

    BOUNDARY = [
        {'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'inner_conductor'},
        {'type': 'Dirichlet', 'tag': 102, 'value': 1.0, 'name': 'outer_conductor'}]

    MATERIAL = [
        {'type': 'relative_electric_permittivity', 'tag': 201, 'value': 2, 'name': 'medium_1'},
        {'type': 'relative_electric_permittivity', 'tag': 202, 'value': 4, 'name': 'medium_2'}]
    
    # Run the domain geometry
    domain.get_coaxial_geometry(element_type, BOUNDARY, MATERIAL, lc, auto_save=False)
    
    # Create mesh Structure Data from gmsh
    mesh_data['nodes_data'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['conn_data'] = read_mesh.get_conn_data(MATERIAL)
    gmsh.finalize()    

    # Get FEM solution
    rho = np.zeros(len(mesh_data['conn_data']))
    Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, rho, element_type)
    ur = spsolve(Agr.tocsr(), fgr.toarray())
    u = boundary_conditions.global_potential_vector(mesh_data, ur)
    return u, mesh_data

# `convergence()`

In [7]:
V0 = 1.0  # Diferença de potencial (V)
analytical_solution = lambda x, y: potential_and_gradient(x, y)
mesh_sizes = [0.5E-2, 1E-3, 0.5E-3]
element_types = [('Triangle', 1), ('Triangle', 2), ('Triangle', 3)]

# Inicialização dos dicionários para armazenar erros e capacitâncias
errors_l2 = {et: [] for et in element_types}
errors_energy = {et: [] for et in element_types}
capacitances = {et: [] for et in element_types}

for lc in mesh_sizes:  # Loop para cada refinamento de malha
    for element_type in element_types:  # Loop para cada tipo de elemento
        # Define o tipo de problema 
        uh, mesh_data = setTypeProblem(element_type, lc)  
        
        # Calcula o erro L2, o erro de energia e a energia total
        l2, energy, total_energy = error_evaluation.get_errors_and_energy(
            mesh_data, element_type, uh, analytical_solution)

        # Capacitância calculada a partir da energia total
        capacitance = 2 * total_energy / V0**2

        # Acumula os resultados nos dicionários correspondentes
        errors_l2[element_type].append(l2)
        errors_energy[element_type].append(energy)
        capacitances[element_type].append(capacitance)

# Graphics

In [8]:
graph_results.plot_convergence_with_slope(mesh_sizes, errors_l2, model='coaxial', name='L2')
graph_results.plot_convergence_with_slope(mesh_sizes, errors_energy, model='coaxial', name='Energy')
graph_results.capacitance_convergence(mesh_sizes, capacitances)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\2_coaxial_domain\pos_processing\pictures\coaxial_convergence_L2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\2_coaxial_domain\pos_processing\pictures\coaxial_convergence_Energy.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\2_coaxial_domain\pos_processing\pictures\coaxial_capacitance_convergence.svg


### Figure 1: Error Convergence for $L_2$ norm
<figure>
    <img src="pos_processing/pictures/coaxial_convergence_L2.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Convergence of the numerical solution for $L_2$ norm of the error.</figcaption>
</figure>

### Figure 2: Error Convergence for _Energy_ norm
<figure>
    <img src="pos_processing/pictures/coaxial_convergence_Energy.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Convergence of the numerical solution for Energy norm of the error.</figcaption>
</figure>

### Figure 3: Capacitance Convergence
<figure>
    <img src="pos_processing/pictures/coaxial_capacitance_convergence.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 3: Convergence of the numerical solution for Capacitance.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.